# 📊 商品カテゴリごとの売上最大化分析: フラグ別売上比較 TOP100

各カテゴリでフラグ（休日、天気など）がどのように売上に影響するかを分析します。

In [ ]:
# ライブラリインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
import matplotlib.font_manager as fm
JP_FONT_PATH = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
if Path(JP_FONT_PATH).exists():
    JP_FP = fm.FontProperties(fname=JP_FONT_PATH)
    plt.rcParams['font.family'] = JP_FP.get_name()
else:
    print("⚠️ 日本語フォントが見つかりません")

print("=" * 80)
print("📊 商品カテゴリごとの売上最大化分析: フラグ別売上比較 TOP100")
print("=" * 80)

In [ ]:
# データ読み込み
input_file = Path("output/06_final_enriched_20250701_20250930.csv")
print(f"📁 データ読み込み中: {input_file}")
df = pd.read_csv(input_file)
print(f"✅ データ形状: {df.shape}")
print(f"列: {list(df.columns[:10])}...")

In [ ]:
# カテゴリ・商品・売上列の自動検出
category_candidates = [col for col in df.columns if any(x in col for x in ['カテゴリ', 'category', '分類', 'フェイスくくり'])]
sales_candidates = [col for col in df.columns if any(x in col for x in ['売上', '金額', 'sales', 'amt'])]
qty_candidates = [col for col in df.columns if any(x in col for x in ['数量', 'qty', 'quantity', '個数'])]

# カテゴリ列の優先順位: 大分類 > 中分類 > 小分類 > カテゴリ
if 'フェイスくくり大分類' in df.columns:
    category_col = 'フェイスくくり大分類'
elif 'フェイスくくり中分類' in df.columns:
    category_col = 'フェイスくくり中分類'
elif 'フェイスくくり小分類' in df.columns:
    category_col = 'フェイスくくり小分類'
elif 'カテゴリ' in df.columns:
    category_col = 'カテゴリ'
else:
    category_col = category_candidates[0] if category_candidates else None

# 売上列の検出（優先順位: 売上金額 > その他）
if '売上金額' in df.columns:
    sales_col = '売上金額'
elif sales_candidates:
    sales_col = sales_candidates[0]
else:
    sales_col = None

# 数量列の検出
if '売上数量' in df.columns:
    qty_col = '売上数量'
elif qty_candidates:
    qty_col = qty_candidates[0]
else:
    qty_col = None

print(f"✅ カテゴリ列: {category_col}")
print(f"✅ 売上列: {sales_col}")
print(f"✅ 数量列: {qty_col}")
print(f"\nユニークカテゴリ数: {df[category_col].nunique() if category_col else 0}")

In [ ]:
# フラグ列の特定
flag_cols = [col for col in df.columns if 'フラグ' in col and df[col].dtype in ['int64', 'float64', 'bool']]
print(f"🚩 フラグ列数: {len(flag_cols)}")
for flag in flag_cols:
    print(f"  - {flag}")

In [ ]:
# カテゴリごとのフラグ別売上影響を計算
print("🔍 カテゴリごとのフラグ別売上影響を計算中...")

results = []

for category in df[category_col].unique():
    if pd.isna(category):
        continue
    
    cat_data = df[df[category_col] == category].copy()
    
    if len(cat_data) < 10:
        continue
    
    for flag_col in flag_cols:
        flag_on_data = cat_data[cat_data[flag_col] == 1]
        flag_off_data = cat_data[cat_data[flag_col] == 0]
        
        if len(flag_on_data) < 3 or len(flag_off_data) < 3:
            continue
        
        flag_on_sales = flag_on_data[sales_col].mean()
        flag_off_sales = flag_off_data[sales_col].mean()
        sales_diff = flag_on_sales - flag_off_sales
        impact_rate = (sales_diff / flag_off_sales * 100) if flag_off_sales > 0 else 0
        
        qty_impact = None
        if qty_col:
            flag_on_qty = flag_on_data[qty_col].mean()
            flag_off_qty = flag_off_data[qty_col].mean()
            qty_diff = flag_on_qty - flag_off_qty
            qty_impact = (qty_diff / flag_off_qty * 100) if flag_off_qty > 0 else 0
        
        results.append({
            'カテゴリ': category,
            'フラグ': flag_col,
            'フラグON時売上': flag_on_sales,
            'フラグOFF時売上': flag_off_sales,
            '売上差分': sales_diff,
            '売上影響率(%)': impact_rate,
            'フラグON日数': len(flag_on_data),
            'フラグOFF日数': len(flag_off_data),
            '数量影響率(%)': qty_impact,
            '絶対影響額': abs(sales_diff)
        })

results_df = pd.DataFrame(results)
print(f"✅ 分析結果: {len(results_df)}行")

In [ ]:
# TOP100の抽出
top100 = results_df.nlargest(100, '絶対影響額').copy()
print("\n🏆 売上影響TOP100:")
display(top100[['カテゴリ', 'フラグ', '売上影響率(%)', '売上差分', 'フラグON日数']].head(30))
print(f"\n... (全{len(top100)}件のTOP100データ)")

In [ ]:
# 可視化1: プラス/マイナス影響TOP20ヒートマップ（変更なし）
fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# プラス影響TOP20
top_positive = results_df.nlargest(20, '売上影響率(%)')
pivot_positive = top_positive.pivot_table(
    index='カテゴリ',
    columns='フラグ',
    values='売上影響率(%)',
    fill_value=0
)

sns.heatmap(pivot_positive, annot=True, fmt='.1f', cmap='Reds',
            cbar_kws={'label': '売上影響率(%)'}, ax=axes[0])
axes[0].set_title('売上プラス影響 TOP20カテゴリ×フラグ', fontproperties=JP_FP, fontsize=14, pad=20)
axes[0].set_xlabel('フラグ', fontproperties=JP_FP)
axes[0].set_ylabel('カテゴリ', fontproperties=JP_FP)

# マイナス影響TOP20
top_negative = results_df.nsmallest(20, '売上影響率(%)')
pivot_negative = top_negative.pivot_table(
    index='カテゴリ',
    columns='フラグ',
    values='売上影響率(%)',
    fill_value=0
)

sns.heatmap(pivot_negative, annot=True, fmt='.1f', cmap='Blues_r',
            cbar_kws={'label': '売上影響率(%)'}, ax=axes[1])
axes[1].set_title('売上マイナス影響 TOP20カテゴリ×フラグ', fontproperties=JP_FP, fontsize=14, pad=20)
axes[1].set_xlabel('フラグ', fontproperties=JP_FP)
axes[1].set_ylabel('カテゴリ', fontproperties=JP_FP)

plt.tight_layout()
plt.savefig('output/category_flag_impact_heatmap_top20.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 可視化2: TOP100バーチャート（上位50件と下位50件に分けて表示）
print("\n📊 TOP100の可視化（上位50件 + 下位50件）")

# 上位50件
fig, ax = plt.subplots(figsize=(14, 20))

top50_sorted = top100.nlargest(50, '売上差分').sort_values('売上差分', ascending=True)
colors = ['red' if x > 0 else 'blue' for x in top50_sorted['売上差分']]

y_labels = [f"{row['カテゴリ'][:20]}\n({row['フラグ']})"
            for _, row in top50_sorted.iterrows()]

ax.barh(range(len(top50_sorted)), top50_sorted['売上差分'], color=colors, alpha=0.7)
ax.set_yticks(range(len(top50_sorted)))
ax.set_yticklabels(y_labels, fontproperties=JP_FP, fontsize=8)
ax.set_xlabel('売上差分（円）', fontproperties=JP_FP, fontsize=12)
ax.set_title('カテゴリ×フラグ別 売上影響TOP50（1位〜50位）', fontproperties=JP_FP, fontsize=14, pad=20)
ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax.grid(axis='x', alpha=0.3)

# 値ラベル追加
for i, (_, row) in enumerate(top50_sorted.iterrows()):
    value = row['売上差分']
    label = f"¥{value:,.0f}\n({row['売上影響率(%)']:.1f}%)"
    x_pos = value + (max(top50_sorted['売上差分']) * 0.02 if value > 0 else min(top50_sorted['売上差分']) * 0.02)
    ax.text(x_pos, i, label, va='center', fontsize=7, fontproperties=JP_FP)

plt.tight_layout()
plt.savefig('output/category_flag_impact_top50_bar.png', dpi=300, bbox_inches='tight')
plt.show()

# 下位50件（51位〜100位）
fig, ax = plt.subplots(figsize=(14, 20))

bottom50_sorted = top100.nsmallest(50, '売上差分').sort_values('売上差分', ascending=True)
colors = ['red' if x > 0 else 'blue' for x in bottom50_sorted['売上差分']]

y_labels = [f"{row['カテゴリ'][:20]}\n({row['フラグ']})"
            for _, row in bottom50_sorted.iterrows()]

ax.barh(range(len(bottom50_sorted)), bottom50_sorted['売上差分'], color=colors, alpha=0.7)
ax.set_yticks(range(len(bottom50_sorted)))
ax.set_yticklabels(y_labels, fontproperties=JP_FP, fontsize=8)
ax.set_xlabel('売上差分（円）', fontproperties=JP_FP, fontsize=12)
ax.set_title('カテゴリ×フラグ別 売上影響51位〜100位', fontproperties=JP_FP, fontsize=14, pad=20)
ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax.grid(axis='x', alpha=0.3)

# 値ラベル追加
for i, (_, row) in enumerate(bottom50_sorted.iterrows()):
    value = row['売上差分']
    label = f"¥{value:,.0f}\n({row['売上影響率(%)']:.1f}%)"
    x_pos = value + (max(bottom50_sorted['売上差分']) * 0.02 if value > 0 else min(bottom50_sorted['売上差分']) * 0.02)
    ax.text(x_pos, i, label, va='center', fontsize=7, fontproperties=JP_FP)

plt.tight_layout()
plt.savefig('output/category_flag_impact_51to100_bar.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 可視化3: フラグ別影響分布
fig, ax = plt.subplots(figsize=(14, 8))

flag_summary = results_df.groupby('フラグ').agg({
    '売上影響率(%)': ['mean', 'std', 'count'],
    '売上差分': 'sum'
}).round(2)

flag_summary.columns = ['平均影響率(%)', '標準偏差', 'カテゴリ数', '総影響額']
flag_summary = flag_summary.sort_values('平均影響率(%)', ascending=False)

x = range(len(flag_summary))
ax.bar(x, flag_summary['平均影響率(%)'], alpha=0.7, color='steelblue')
ax.set_xticks(x)
ax.set_xticklabels(flag_summary.index, rotation=45, ha='right', fontproperties=JP_FP)
ax.set_ylabel('平均売上影響率(%)', fontproperties=JP_FP, fontsize=12)
ax.set_title('フラグ別 平均売上影響率（全カテゴリ）', fontproperties=JP_FP, fontsize=14, pad=20)
ax.axhline(y=0, color='red', linestyle='--', linewidth=1)
ax.grid(axis='y', alpha=0.3)

for i, (idx, row) in enumerate(flag_summary.iterrows()):
    ax.text(i, row['平均影響率(%)'] + 0.5,
            f"{row['平均影響率(%)']:.1f}%\n(n={int(row['カテゴリ数'])})",
            ha='center', va='bottom', fontsize=9, fontproperties=JP_FP)

plt.tight_layout()
plt.savefig('output/flag_impact_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 結果保存
top100.to_csv('output/category_flag_analysis_top100.csv', index=False, encoding='utf-8-sig')
results_df.to_csv('output/category_flag_analysis_all.csv', index=False, encoding='utf-8-sig')
print("✅ 結果をCSV保存しました")
print(f"   - output/category_flag_analysis_top100.csv ({len(top100)}件)")
print(f"   - output/category_flag_analysis_all.csv ({len(results_df)}件)")

In [ ]:
# サマリーレポート
print("\n" + "=" * 60)
print("📊 分析サマリー")
print("=" * 60)

print(f"\n総カテゴリ数: {df[category_col].nunique()}")
print(f"分析対象フラグ数: {len(flag_cols)}")
print(f"分析結果総数: {len(results_df)}行")

print("\n🏆 最大プラス影響:")
max_positive = results_df.loc[results_df['売上影響率(%)'].idxmax()]
print(f"  カテゴリ: {max_positive['カテゴリ']}")
print(f"  フラグ: {max_positive['フラグ']}")
print(f"  影響率: +{max_positive['売上影響率(%)']:.1f}%")
print(f"  売上差分: ¥{max_positive['売上差分']:,.0f}")

print("\n📉 最大マイナス影響:")
max_negative = results_df.loc[results_df['売上影響率(%)'].idxmin()]
print(f"  カテゴリ: {max_negative['カテゴリ']}")
print(f"  フラグ: {max_negative['フラグ']}")
print(f"  影響率: {max_negative['売上影響率(%)']:.1f}%")
print(f"  売上差分: ¥{max_negative['売上差分']:,.0f}")

print("\n💡 推奨アクション:")
print("1. プラス影響の大きいカテゴリ×フラグの組み合わせで販促強化")
print("2. マイナス影響の大きい組み合わせで在庫調整・仕入れ最適化")
print("3. フラグON日数が少ない組み合わせはデータ蓄積が必要")